In [2]:
#Importing Necesarry Modules
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pandas as pd


In [3]:
from selenium.common.exceptions import TimeoutException

def wait_for_page_to_load(driver, wait):
    title = driver.title
    try:
        wait.until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )
    except TimeoutException:
        print(f"The page '{title}' did not get fully loaded.\n")
    else:
        print(f"The page '{title}' did get fully loaded.\n")


In [4]:
options=Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
'''
options.add_argument("--disable-http2")
options.add_argument("--incognito")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6422.113 Safari/537.36"

)'''

driver=webdriver.Chrome(options=options)
driver.maximize_window()

#Explicit wait
wait=WebDriverWait(driver,20)

#Accessing The Url
url="https://www.99acres.com"
driver.get(url)
wait_for_page_to_load(driver,wait)

#Identify and enter text into searchbar
try:
    search_bar=wait.until(
        EC.element_to_be_clickable((By.XPATH,'//*[@id="keyword2"]'))
    )
except:
    print("Timeout while clicking on \"Search\" button.\n")
else:
    search_bar.send_keys("Chennai")
    time.sleep(2)

#selecting valid option
try:
    valid_option=wait.until(
    EC.element_to_be_clickable((By.XPATH,'//*[@id="0"]'))
    )
except:
    print("Timeout while locating valid search option")
else:
    valid_option.click()
    time.sleep(2)

#click on Search Button
try:
    search_button=wait.until(
        EC.element_to_be_clickable((By.XPATH,'//*[@id="searchform_search_btn"]'))
    )
except:
    print("Timeout while clicking on the search button")
else:
    search_button.click()
    wait_for_page_to_load(driver,wait)

#Adjust the budget slider
try:
    slider=wait.until(
        EC.element_to_be_clickable((By.XPATH,'//*[@id="budgetLeftFilter_max_node"]'))
    )
except:
        print("Timeout while clicking on the budget slider node")
else:
    actions=ActionChains(driver)
    (
        actions
        .click_and_hold(slider)
        .move_by_offset(-73,0)
        .release()
        .perform()
    )
    time.sleep(2)

#filter result
verified=wait.until(
    EC.element_to_be_clickable((By.XPATH,'/html[1]/body[1]/div[1]/div[1]/div[1]/div[4]/div[3]/div[1]/div[3]/section[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[3]/span[1]'))
)
verified.click()
time.sleep(1)

ready_to_move=wait.until(
    EC.element_to_be_clickable((By.XPATH,"/html[1]/body[1]/div[1]/div[1]/div[1]/div[4]/div[3]/div[1]/div[3]/section[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[5]/span[1]"))
)
ready_to_move.click()
time.sleep(1)

for _ in range(5):
    try:
        filter_right_button=wait.until(
            EC.presence_of_element_located(By.XPATH,"//i[@class='iconS_Common_24 icon_upArrow cc__rightArrow']")
        )
    except:
        print("Timeout we have uncoveered all the filters")
        break
    else:
        filter_right_button.click()
#with photos
with_photos=wait.until(
    EC.element_to_be_clickable((By.XPATH,"//span[normalize-space()='With Photos']"))
)
with_photos.click()
time.sleep(2)


#NAVIGTE PAGE AND EXTRACTS DATA
data=[]
page_count=0
while True:
    page_count+=1
    try:
        next_page_button=driver.find_element(By.XPATH,"//a[normalize-space()='Next Page >']")
    except:
        print(f"We have navigated all the {page_count} pages!\n")
        break
    
    else:

        try:

            driver.execute_script("window.scrollBy(0,arguments[0].getBoundingClientRect().top -100);",next_page_button)
            time.sleep(3)

            #SCAPPING THE DATA
            rows=driver.find_elements(By.CLASS_NAME,'tupleNew__contentWrap')
            for row in rows:
                #PROPERTY NAME
                try:
                    name=row.find_element(By.CLASS_NAME,"tupleNew__headingNrera").text
                except:
                    name=np.nan

                #PROPERTY LOCATION
                try:
                    location=row.find_element(By.CLASS_NAME,"tupleNew__propType").text
                except:
                    location=np.nan

                #PROPERTY PRICE
                try:
                    price=row.find_element(By.CLASS_NAME,"tupleNew__priceValWrap").text
                except:
                    price=np.nan
            
                #AREA ELEMENTS
                try:
                    elements=row.find_elements(By.CLASS_NAME,'tupleNew__area1Type')
                except:
                    area,bhk=[np.nan,np.nan]
                else:
                    area,bhk=[ele.text for ele in elements]

                property={
                    "name":name,
                    "location":location,
                    "price":price,
                    "area":area,
                    "BHK":bhk
                }
                data.append(property)
                
            wait.until(
            EC.element_to_be_clickable((By.XPATH,"//a[normalize-space()='Next Page >']"))
            )
            next_page_button.click()
            time.sleep(5)
        except:
            print("Timeout while clicking on next page")

#SCRAPPING DATA FROM THE LAST PAGE
rows = driver.find_elements(By.CLASS_NAME, 'tupleNew__contentWrap')
for row in rows:
    # PROPERTY NAME
    try:
        name = row.find_element(By.CLASS_NAME, "tupleNew__headingNrera").text
    except:
        name = np.nan

    # PROPERTY LOCATION
    try:
        location = row.find_element(By.CLASS_NAME, "tupleNew__propType").text
    except:
        location = np.nan

    # PROPERTY PRICE
    try:
        price = row.find_element(By.CLASS_NAME, "tupleNew__priceValWrap").text
    except:
        price = np.nan

    # AREA ELEMENTS
    try:
        elements = row.find_elements(By.CLASS_NAME, 'tupleNew__area1Type')
        area, bhk = [ele.text for ele in elements]
    except:
        area, bhk =[np.nan, np.nan]

    property = {
        "name": name,
        "location": location,
        "price": price,
        "area": area,
        "BHK": bhk
    }

    data.append(property)

time.sleep(5)
driver.quit()


The page 'India Real Estate Property Site - Buy Sell Rent Properties Portal - 99acres.com' did get fully loaded.

The page 'Property in Chennai - Real Estate in Chennai' did get fully loaded.

Timeout we have uncoveered all the filters
Timeout while clicking on next page
Timeout while clicking on next page
We have navigated all the 83 pages!



In [5]:
len(data)

2308

In [7]:
data

[{'name': "Dharman's Divine",
  'location': '2 BHK Flat in Sankarapuram, Sithalapakkam',
  'price': '₹57.7 Lac',
  'area': '1,045 sqft',
  'BHK': '2 BHK'},
 {'name': 'ENCLAVE',
  'location': '3 BHK Flat in Valasaravakkam, Chennai',
  'price': '₹1.22 Cr',
  'area': '1,380 sqft',
  'BHK': '3 BHK'},
 {'name': 'GRN East Avenue',
  'location': '3 BHK Flat in Korattur, Chennai',
  'price': '₹1.55 Cr',
  'area': '1,405 sqft',
  'BHK': '3 BHK'},
 {'name': 'Bharathi Nagar,Ayanavaram, Chennai, Chennai North',
  'location': '6 Bedroom House in Bharathi Nagar,Ayanavaram, Chennai',
  'price': '₹2.5 Cr',
  'area': '820 sqft',
  'BHK': '6 BHK'},
 {'name': 'Shrusti Vedanshi',
  'location': '2 BHK Flat in Velachery, Chennai',
  'price': '₹1.75 Cr',
  'area': '975 sqft',
  'BHK': '2 BHK'},
 {'name': 'Purasaiwakkam, Chennai, Chennai Central',
  'location': '6 Bedroom House in Purasaiwakkam, Chennai',
  'price': '₹1.15 Cr',
  'area': '1,197 sqft',
  'BHK': '6 BHK'},
 {'name': 'Radiance Sapphire',
  'locat

In [14]:
#Data Cleaning
df_properties=(
    pd
    .DataFrame(data)
    .drop_duplicates()
    .apply(lambda col:col.str.strip().str.lower() if col.dtype=="object" else col)
    .assign(
        is_starred=lambda df_:df_.name.str.contains("\n").astype(int),
        name=lambda df_:(
            df_.name.str.replace("\n[0-9.]+"," ",regex=True)
            ),
        location=lambda df_:(
            df_.location
            .str.replace("chennai","")
            .str.strip()
            .str.replace(",$","",regex=True)
            .str.split("in").str[-1].str.strip()
        ),
        price= lambda df_:(
            df_
            .price
            .str.replace("?","")
            .apply(lambda val :val.replace("lac","").strip() if "lac" in val else val.replace("cr","").strip() * 100)
        ),
        area=lambda df_:(
            df_
            .area
            .str.replace("sqft","")
            .str.replace(",","")
            .pipe(lambda ser:pd.to_numeric(ser))
        ),
        BHK=lambda df_:(
            df_
            .BHK
            .str.replace("bhk","")
            .str.strip()
            .pipe(lambda ser:pd.to_numeric(ser))
        )

    )
    .rename(columns={
        "price":"price in lakh",
        "area":"area_sqft"
    })
    .reset_index(drop=True)
    .to_excel("Chennai-properties-99acres.xlsx",index=False)
)


df_properties